<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python---PyODBC" data-toc-modified-id="Python---PyODBC-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python - PyODBC</a></span><ul class="toc-item"><li><span><a href="#Installation-and-Imports" data-toc-modified-id="Installation-and-Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Installation and Imports</a></span></li><li><span><a href="#Example:-Use-pyodbc-to-create,-write-and-read-from-a-sqlite-database-file" data-toc-modified-id="Example:-Use-pyodbc-to-create,-write-and-read-from-a-sqlite-database-file-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Example: Use pyodbc to create, write and read from a sqlite database file</a></span><ul class="toc-item"><li><span><a href="#Establish-a-pyodbc-connection" data-toc-modified-id="Establish-a-pyodbc-connection-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Establish a pyodbc connection</a></span></li><li><span><a href="#Interact-with-the-database-with-SQL-queries" data-toc-modified-id="Interact-with-the-database-with-SQL-queries-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Interact with the database with SQL queries</a></span><ul class="toc-item"><li><span><a href="#Create-a-table-to-store-some-data" data-toc-modified-id="Create-a-table-to-store-some-data-1.2.2.1"><span class="toc-item-num">1.2.2.1&nbsp;&nbsp;</span>Create a table to store some data</a></span></li><li><span><a href="#Insert-data-into-this-table" data-toc-modified-id="Insert-data-into-this-table-1.2.2.2"><span class="toc-item-num">1.2.2.2&nbsp;&nbsp;</span>Insert data into this table</a></span></li><li><span><a href="#Connect-to-this-database-again-and-fetch-the-phone-column-from-the-table" data-toc-modified-id="Connect-to-this-database-again-and-fetch-the-phone-column-from-the-table-1.2.2.3"><span class="toc-item-num">1.2.2.3&nbsp;&nbsp;</span>Connect to this database again and fetch the phone column from the table</a></span></li><li><span><a href="#Connect-to-this-database-and-fetch-all-the-rows-one-at-a-time" data-toc-modified-id="Connect-to-this-database-and-fetch-all-the-rows-one-at-a-time-1.2.2.4"><span class="toc-item-num">1.2.2.4&nbsp;&nbsp;</span>Connect to this database and fetch all the rows one at a time</a></span></li></ul></li></ul></li></ul></li></ul></div>

---
# Python - PyODBC
---
PyODBC is an open-source Python library for accessing database systems. ODBC stands for Open Database Connectivity and it provides a standard interface to connect between databases and other applications (e.g. a Python program). 

In this notebook, we will use PyODBC to interface with a simple SQLite database. PyODBC can also be used to interface with other databases. For example, 
- an MS Excel or MS Access file
- a local MSSQL server
- a remote SQL server on an AWS instance

PyODBC works in the following manner:
1) Establish a connection (typically with authentication) to the database server using `pyodbc.connect`  
2) Create a cursor object to interact with the database  
3) Execute queries using this cursor object  
4) Fetch or commit the results based on the queries
5) Close the connection

## Installation and Imports
PyODBC can be installed using pip or conda as follows:  
`pip install pyodbc` or `conda install pyodbc`

In [1]:
# Import
import pyodbc

In [2]:
# List the drivers installed and available for use with pyodbc
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'ODBC Driver 17 for SQL Server',
 'SQL Server Native Client RDA 11.0',
 'SQLite3 ODBC Driver',
 'SQLite ODBC Driver',
 'SQLite ODBC (UTF-8) Driver',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

If SQLite3/SQLite ODBC Drivers are not listed, they can be installed from <http://www.ch-werner.de/sqliteodbc/> (sqliteodbc_w64.exe). 

Note: Python provides a default SQLite interface using the built-in sqlite3 library. Here, we are connecting using pyodbc's sqlite drivers to illustrate the use of pyodbc to interface with different database management systems. 

## Example: Use pyodbc to create, write and read from a sqlite database file
In this example, we create a database file `addressbook.db` and write a simple table into this database and query from this table. 

### Establish a pyodbc connection 
We use the appropriate driver (in this example, "SQLite3 ODBC Driver") to establish the connection.

In [3]:
conn = pyodbc.connect(r'DRIVER=SQLite3 ODBC Driver;'
                      r'DATABASE=addressbook.db')

Note: For connecting to other database systems, you may have to include additional parameters such as:
- `SERVER` (could be either localhost or a remote cloud instance)  
- `PORT` (the server port to connect to)
- `UID` (Username to login)
- `PWD` (Password for authentication)

For more information on how to connect to different databases, see <https://github.com/mkleehammer/pyodbc/wiki>

In [4]:
# Create a cursor object
cursor = conn.cursor()

### Interact with the database with SQL queries 

#### Create a table to store some data

In [5]:
# First, the SQL query is constructed as a Python string
query = '''
create table addressbook (
    first_name varchar(20),
    last_name varchar(20),
    city varchar(20),
    phone varchar(15)
)
'''

In [6]:
# Call the cursor.execute() method to perform the SQL query
cursor.execute(query)

#### Insert data into this table
In order to use values from Python variables (dynamic parameters in a query), we can use question marks as placeholders (qmark style). This is probably the easiest/cleanest way to execute queries. Using Python's string operations directly to construct a query is insecure and can be susceptible to SQL injection attacks.  

Inserting one row into the table using `cursor.execute()` in qmark style

In [7]:
# Data to be inserted
jsmith_address = ('John','Smith','London','+442012345678')

# Execute the query and commit the changes
cursor.execute("insert into addressbook values (?,?,?,?)",jsmith_address)
cursor.commit()

Inserting  multiple rows into the table using `cursor.executemany()` in qmark style

In [8]:
# Data to be inserted
address_list = [('Jane','Doe','Belfast','+449012345678'),
               ('Joe','Bloggs','New York','+12121234567')]

#Execute the query and commit the changes
cursor.executemany("insert into addressbook values (?,?,?,?)", address_list)
cursor.commit()

In [9]:
# Close the connection after running the SQL commands
conn.close()

#### Connect to this database again and fetch the phone column from the table
We can fetch multiple results at the same time using `cursor.fetchall()` method

In [10]:
# Establish the connection to the database file
conn = pyodbc.connect(r"Driver=SQLite3 ODBC Driver;Database=addressbook.db")
cursor = conn.cursor()

In [11]:
# Execute the SQL query and fetch the results
query = '''select phone from addressbook'''

cursor.execute(query)

results = cursor.fetchall()
conn.close()

In [12]:
# Print the results
print(results)

[('+442012345678', ), ('+449012345678', ), ('+12121234567', )]


#### Connect to this database and fetch all the rows one at a time
We can fetch one result at a time using `cursor.fetchone()` method

In [13]:
# Establish the connection to the file
conn = pyodbc.connect(r"DRIVER=SQLite3 ODBC Driver;DATABASE=addressbook.db")
cursor = conn.cursor()

In [14]:
# Execute the query and fetch the results
query = '''select * from addressbook'''

cursor.execute(query)

result = True
while result is not None:
    result = cursor.fetchone()
    print(result)
    
results = cursor.fetchall()
conn.close()

('John', 'Smith', 'London', '+442012345678')
('Jane', 'Doe', 'Belfast', '+449012345678')
('Joe', 'Bloggs', 'New York', '+12121234567')
None


#### Concept Check <a class='tocSkip'>
Use PyODBC to add a Country column to this table and fetch details of people in United Kingdom.

In [ ]:
# Type your code here








